# Kaggle: San Francisco Crime Classification
## Improvement as part of sdap17 excercise 3

In [1]:
import pandas as pd
import numpy as np
import pprint
import requests

## Exploration of the training data set

In [3]:
train_data = pd.read_csv("../../data/raw/train.csv")
train_data['Dates'] = pd.to_datetime(train_data['Dates'])

In [4]:
len(train_data)

878049

In [5]:
train_data.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [6]:
crimes = train_data['Category'].unique()
pprint.pprint("Crimes: {}, #{}".format(crimes, len(crimes)), indent=2)

("Crimes: ['WARRANTS' 'OTHER OFFENSES' 'LARCENY/THEFT' 'VEHICLE THEFT' "
 "'VANDALISM'\n"
 " 'NON-CRIMINAL' 'ROBBERY' 'ASSAULT' 'WEAPON LAWS' 'BURGLARY'\n"
 " 'SUSPICIOUS OCC' 'DRUNKENNESS' 'FORGERY/COUNTERFEITING' 'DRUG/NARCOTIC'\n"
 " 'STOLEN PROPERTY' 'SECONDARY CODES' 'TRESPASS' 'MISSING PERSON' 'FRAUD'\n"
 " 'KIDNAPPING' 'RUNAWAY' 'DRIVING UNDER THE INFLUENCE'\n"
 " 'SEX OFFENSES FORCIBLE' 'PROSTITUTION' 'DISORDERLY CONDUCT' 'ARSON'\n"
 " 'FAMILY OFFENSES' 'LIQUOR LAWS' 'BRIBERY' 'EMBEZZLEMENT' 'SUICIDE'\n"
 " 'LOITERING' 'SEX OFFENSES NON FORCIBLE' 'EXTORTION' 'GAMBLING'\n"
 " 'BAD CHECKS' 'TREA' 'RECOVERED VEHICLE' 'PORNOGRAPHY/OBSCENE MAT'], #39")


In [7]:
train_data['Category'].value_counts()

LARCENY/THEFT                  174900
OTHER OFFENSES                 126182
NON-CRIMINAL                    92304
ASSAULT                         76876
DRUG/NARCOTIC                   53971
VEHICLE THEFT                   53781
VANDALISM                       44725
WARRANTS                        42214
BURGLARY                        36755
SUSPICIOUS OCC                  31414
MISSING PERSON                  25989
ROBBERY                         23000
FRAUD                           16679
FORGERY/COUNTERFEITING          10609
SECONDARY CODES                  9985
WEAPON LAWS                      8555
PROSTITUTION                     7484
TRESPASS                         7326
STOLEN PROPERTY                  4540
SEX OFFENSES FORCIBLE            4388
DISORDERLY CONDUCT               4320
DRUNKENNESS                      4280
RECOVERED VEHICLE                3138
KIDNAPPING                       2341
DRIVING UNDER THE INFLUENCE      2268
RUNAWAY                          1946
LIQUOR LAWS 

Generiere zeitbasierte Features wie in Präsentation vorgestellt.

In [48]:
def get_halfhour(minute):
    if minute < 30:
        return 0
    else:
        return 1

def get_daynight(hour):
    if 5 < hour and hour < 23:
        return 0
    else:
        return 1
    
def generate_time_features(times):
    minute_series = pd.Series([x.minute for x in times], name='minute')
    halfhour_series = pd.Series([get_halfhour(x.minute) for x in times], name='halfhour')
    hour_series = pd.Series([x.hour for x in times], name='hour')
    daynight_series = pd.Series([get_daynight(x.hour) for x in times], name='day_night')
    day_series = pd.Series([x.day for x in times], name='day')
    month_series = pd.Series([x.month for x in times], name='month')
    year_series = pd.Series([x.year for x in times], name='year')
    
    time_features = pd.concat([minute_series, halfhour_series, hour_series, daynight_series, day_series, month_series, year_series], axis=1)
    return time_features

In [49]:
times = train_data["Dates"]

In [50]:
time_features = generate_time_features(times)
print("success")

success


In [51]:
print(time_features)

        minute  halfhour  hour  day_night  day  month  year
0           53         1    23          1   13      5  2015
1           53         1    23          1   13      5  2015
2           33         1    23          1   13      5  2015
3           30         1    23          1   13      5  2015
4           30         1    23          1   13      5  2015
5           30         1    23          1   13      5  2015
6           30         1    23          1   13      5  2015
7           30         1    23          1   13      5  2015
8            0         0    23          1   13      5  2015
9            0         0    23          1   13      5  2015
10          58         1    22          0   13      5  2015
11          30         1    22          0   13      5  2015
12          30         1    22          0   13      5  2015
13           6         0    22          0   13      5  2015
14           0         0    22          0   13      5  2015
15           0         0    22          